# Setup

 Import standard modules

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
import numpy as np
import pandas as pd
import time
import seaborn as sns
sns.set()
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor
from scipy.signal import argrelmin, argrelmax
from scipy.interpolate import InterpolatedUnivariateSpline, interp1d
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed

os.chdir("..")
from src import features, models
from src import definitions as defs

In [2]:
wdir = defs.ROOT / "data" / "interim"

Import project modules from src folder

In [3]:
def lm_tides(data, x, y, label):
    from collections import namedtuple
    Result = namedtuple("Result", ["lm", "preds", "coef", "intercept", "eq"])
    lm = LinearRegression()
    x = eval("data.{}".format(x))
    x = x.values.reshape(-1, 1)
    y = data[y].values.reshape(-1, 1)
    lm.fit(x, y)
    preds = lm.predict(x).reshape(-1)
    coef = lm.coef_[0][0] 
    intercept = lm.intercept_[0]

    eq = "$\zeta_{{{}}}={:.4f}t {:+.4f}$".format(label, coef, intercept)
    return Result(lm, preds, coef, intercept, eq)

def stretch_limb(limb, old_range: list(), new_range: list()):
    tide = limb.copy()
    tide["elevation"] = new_range[0] + ((tide.elevation - old_range[0]) * (new_range[1] - new_range[0])) / (tide.elevation.max() - old_range[0])
    return tide

In [4]:
MSL = 1.12
beta_mean_high = 14.5e-3
beta_mean_low = -2e-3

In [5]:
tides = features.load_tide(wdir, "tides-50yr-30s.feather")
tides = tides.to_frame(name="elevation")
tides["elapsed_sec"] = (tides.index - tides.index[0]).total_seconds().astype(int)
tides["elapsed_yr"] = tides.elapsed_sec / pd.Timedelta(days=365).total_seconds()
tides = tides[["elapsed_sec", "elapsed_yr", "elevation"]]
lm = lm_tides(tides, x="elapsed_yr", y="elevation", label="MW")
tides["elevation"] = tides.elevation - lm.coef * tides.elapsed_sec / pd.Timedelta(days=365).total_seconds()

In [6]:
tides[["high", "low"]] = False

tides.loc[tides.index[argrelmax(tides.elevation.values)[0]], "high"] = True
tides.loc[tides.index[argrelmin(tides.elevation.values)[0]], "low"] = True

nodes = pd.concat([tides[tides.high==True], tides[tides.low==True]], keys=("high", "low"), names=("type", "datetime"))
nodes = nodes.reset_index(level=["type"]).sort_index()
nodes = nodes[["elapsed_sec", "elapsed_yr", "type", "elevation"]]

mhw = nodes.groupby(by="type").resample("M").mean().loc["high"]
mlw = nodes.groupby(by="type").resample("M").mean().loc["low"]

mhw["elevation_new"] = mhw.elevation + mhw.elapsed_yr * beta_mean_high
mlw["elevation_new"] = mlw.elevation + mlw.elapsed_yr * beta_mean_low

In [7]:
highs = nodes.loc[nodes.type=="high"].reset_index()[["datetime", "elevation"]]
highs = highs.append([{"datetime": (highs.datetime.iat[-1] + highs.datetime.diff().mean()).round("30S")}]).sort_values("datetime").reset_index(drop=True)
highs["elapsed_yr"] = (highs.datetime - tides.index[0]).dt.total_seconds() / pd.Timedelta(days=365).total_seconds()
lows = nodes.loc[nodes.type=="low"].reset_index()[["datetime", "elevation"]]
lows = lows.append([{"datetime": (lows.datetime.iat[0] - lows.datetime.diff().mean()).round("30S")}]).sort_values("datetime").reset_index(drop=True)
lows["elapsed_yr"] = (lows.datetime - tides.index[0]).dt.total_seconds() / pd.Timedelta(days=365).total_seconds()

low_subset = lows.iloc[::2].iloc[1:]
low_interp = interp1d(x=low_subset.elapsed_yr, y=low_subset.elevation, fill_value="extrapolate", kind="linear")
lows.loc[0, "elevation"] = low_interp(lows.elapsed_yr.iat[0])

high_subset = highs.iloc[1::2].iloc[:-1]
high_interp = interp1d(x=high_subset.elapsed_yr, y=high_subset.elevation, fill_value="extrapolate", kind="linear")
highs.loc[highs.index[-1], "elevation"] = high_interp(highs.elapsed_yr.iat[-1])

highs["elevation_new"] = highs.elevation + highs.elapsed_yr * beta_mean_high
lows["elevation_new"] = lows.elevation + lows.elapsed_yr * beta_mean_low

df1 = lows.join(other=highs, lsuffix="_low", rsuffix="_high").drop(columns=["elapsed_yr_high", "elapsed_yr_low"])
df1["type"] = "low-high"

df2 = highs[:-1].join(other=lows.shift(-1)[:-1], lsuffix="_high", rsuffix="_low").drop(columns=["elapsed_yr_high", "elapsed_yr_low"])
df2["type"] = "high-low"

df1.columns = ["start", "low_old", "low_new", "stop", "high_old", "high_new", "type"]
df2.columns = ["start", "high_old", "high_new", "stop", "low_old", "low_new", "type"]


df = pd.concat(objs=[df1, df2]).set_index("start").sort_index().reset_index()

df = df[["start", "stop", "type", "high_old", "high_new", "low_old", "low_new"]]
df.index.name = "tide"

In [9]:
new = Parallel(n_jobs=30)(delayed(stretch_limb)(
    limb=tides.loc[row.start:row.stop],
    old_range=[row.low_old, row.high_old],
    new_range=[row.low_new, row.high_new]
    ) for row in df.itertuples())
tides_new = pd.concat(new).reset_index().drop_duplicates(subset=["datetime"]).set_index("datetime").sort_index()

In [10]:
plt.close("all")
plot_data = df.iloc[0:75]

for row in plot_data.itertuples():
    sns.lineplot(data=tides.loc[row.start:row.stop], x="datetime", y="elevation")
    sns.scatterplot(data=tides.loc[row.start:row.stop].loc[tides.high == True], x="datetime", y="elevation", color="green", marker=".")
    sns.scatterplot(data=tides.loc[row.start:row.stop].loc[tides.low == True], x="datetime", y="elevation", color="red", marker=".")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def idx_max(x):
    return x.idxmax().to_datetime64()

def idx_min(x):
    return x.idxmin().to_datetime64()

def find_extrema(df, window, method):
    fun = "idx_{}".format(method)
    extrema = df.elevation.rolling(window=window).agg([method, eval(fun)])
    extrema[fun] = pd.to_datetime(extrema[fun])
    extrema = extrema.set_index(fun).drop_duplicates()
    extrema.index.name = "datetime"
    extrema.columns = ["elevation"]
    return extrema

In [12]:
from scipy.signal import argrelmin, argrelmax

In [14]:
plt.close("all")

# high_highs = highs[::2]
# low_highs = highs[1::2]

# springs = highs[::2].iloc[argrelmax(data=highs[::2].elevation.values)]
# neaps = highs[1::2].iloc[argrelmin(data=highs[1::2].elevation.values)]

# springs = find_extrema(highs.set_index("datetime"), window, "max").reset_index()
# neaps = find_extrema(highs.set_index("datetime"), window, "min").reset_index()

window=10
highs["rollmean"] = highs.elevation.rolling(window=window, center=True).mean().values

springs = highs.iloc[argrelmax(data=highs.rollmean.values)]
neaps = highs.iloc[argrelmin(data=highs.rollmean.values)]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_axes([0.1, 0.25, 0.75, 0.65])

sns.lineplot(data=highs.loc[highs.datetime < "2020-04-01"], x="datetime", y="elevation", color="cornflowerblue", ax=ax)
sns.lineplot(data=highs.loc[highs.datetime < "2020-04-01"], x="datetime", y="rollmean", color="red", ax=ax)
sns.scatterplot(data=springs.loc[springs.datetime < "2020-04-01"], x="datetime", y="elevation", color="green")
sns.scatterplot(data=neaps.loc[neaps.datetime < "2020-04-01"], x="datetime", y="elevation", color="red")
fig.autofmt_xdate(rotation=45) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.close("all")
fig = plt.figure(figsize=(12, 10))
ax = fig.add_axes([0.1, 0.25, 0.75, 0.65])

t = -10
n = 9
i = df.iloc[t].start
ii = df.iloc[t+n].stop

new = tides_new.loc[i:ii]
old = tides.loc[i:ii]

loc = tides.index.get_loc(old.index[0])

sns.lineplot(ax=ax, x=new.index, y=new.elevation, ls=":", color="tomato", zorder=100)
sns.lineplot(x=old.index, y=old.elevation, color="cornflowerblue", ax=ax, zorder=99)
# sns.scatterplot(x=old.loc[old.high == True].index, y=old.loc[old.high == True].elevation, color="green", label="Highs", ax=ax, zorder=101)
# sns.scatterplot(x=old.loc[old.low == True].index, y=old.loc[old.low == True].elevation, color="red", label="Lows", ax=ax, zorder=102)

# sns.scatterplot(x=new.loc[new.high == True].index, y=new.loc[new.high == True].elevation, color="green", ax=ax, zorder=101)
# sns.scatterplot(x=new.loc[new.low == True].index, y=new.loc[new.low == True].elevation, color="red", ax=ax, zorder=102)

high_diff = new.loc[new.high == True].elevation - old.loc[old.high == True].elevation
low_diff = new.loc[new.low == True].elevation - old.loc[old.low == True].elevation

ax.errorbar(x=old.loc[old.high == True].index, y=old.loc[old.high == True].elevation, yerr=high_diff, marker='o', ls="none", lolims=True, color="green")
ax.errorbar(x=old.loc[old.low == True].index, y=old.loc[old.low == True].elevation, yerr=-low_diff, marker='o', ls="none", uplims=True, color="red")

plt.xticks(rotation=45)
ax.legend(loc='center left', bbox_to_anchor=(1,0.5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [640]:
tides_new.loc[tides_new.index < "2021"]

,elapsed_sec,elapsed_yr,elevation,high,low
datetime,,,,,
2020-01-01 00:00:00+06:00,0,0.000000e+00,-0.764954,False,False
2020-01-01 00:00:30+06:00,30,9.512938e-07,-0.757824,False,False
2020-01-01 00:01:00+06:00,60,1.902588e-06,-0.750682,False,False
2020-01-01 00:01:30+06:00,90,2.853881e-06,-0.743526,False,False
2020-01-01 00:02:00+06:00,120,3.805175e-06,-0.736357,False,False
...,...,...,...,...,...
2020-12-31 23:57:30+06:00,31622250,1.002735e+00,1.257118,False,False
2020-12-31 23:58:00+06:00,31622280,1.002736e+00,1.262054,False,False
2020-12-31 23:58:30+06:00,31622310,1.002737e+00,1.266952,False,False


In [16]:
plt.close("all")
data = tides_new
n = pd.Timedelta(minutes=60)
subset = data.resample(n).first()
data_lm = lm_tides(data=data, x="elapsed_yr", y="elevation", label="MW")
subset_highs = data[data.high == True]
subset_highs_lm = lm_tides(data=subset_highs, x="elapsed_yr", y="elevation", label="MHW")
subset_lows = data[data.low == True]
subset_lows_lm = lm_tides(data=subset_lows, x="elapsed_yr", y="elevation", label="MLW")

fig = plt.figure(figsize=(10, 5))
ax = plt.axes()

sns.scatterplot(ax=ax, x=subset.index, y=subset.elevation, color="cornflowerblue", marker=".", edgecolor=None, alpha=0.3, s=1)
sns.scatterplot(ax=ax, x=subset_highs.index, y=subset_highs.elevation, marker=".", color="darkgreen", edgecolor=None, s=1)
sns.scatterplot(ax=ax, x=subset_lows.index, y=subset_lows.elevation, marker=".", color="darkred", edgecolor=None, s=1)

sns.lineplot(ax=ax, x=subset_highs.index, y=subset_highs_lm.preds, color="green", linestyle="--", zorder=101, label=subset_highs_lm.eq)
sns.lineplot(ax=ax, x=[data.index[0], data.index[-1]], y=[data_lm.preds[0], data_lm.preds[-1]], color="blue", linestyle="--", zorder=100, label=data_lm.eq)
sns.lineplot(ax=ax, x=subset_lows.index, y=subset_lows_lm.preds, color="red", linestyle="--", zorder=102, label=subset_lows_lm.eq)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime', ylabel='elevation'>

In [17]:
plt.close("all")
mhw_lm = lm_tides(data=mhw, x="elapsed_yr", y="elevation_new", label="MHW")
mlw_lm = lm_tides(data=mlw, x="elapsed_yr", y="elevation_new", label="MLW")

fig = plt.figure(figsize=(10,7.5))
sns.scatterplot(x=mhw.index, y=mhw.elevation_new, color="green")
sns.lineplot(x=mhw.index, y=mhw_lm.preds, color="black", ls="--", lw=3, label=mhw_lm.eq)

sns.scatterplot(x=mlw.index, y=mlw.elevation_new, color="red")
sns.lineplot(x=mlw.index, y=mlw_lm.preds, color="black", ls="--", lw=3, label=mlw_lm.eq)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime', ylabel='elevation_new'>

In [46]:
features.load_tide(wdir, "tides-50yr-30s.feather").loc[:"2039-12-31"] + MSL

datetime
2020-01-01 00:00:00+06:00    0.355044
2020-01-01 00:00:30+06:00    0.362173
2020-01-01 00:01:00+06:00    0.369316
2020-01-01 00:01:30+06:00    0.376472
2020-01-01 00:02:00+06:00    0.383640
                               ...   
2039-12-31 23:57:30+06:00    2.393360
2039-12-31 23:58:00+06:00    2.396914
2039-12-31 23:58:30+06:00    2.400437
2039-12-31 23:59:00+06:00    2.403930
2039-12-31 23:59:30+06:00    2.407393
Freq: 30S, Name: elevation, Length: 21038400, dtype: float64

In [29]:
from src import models
ts = features.load_tide(wdir, "tides-50yr-30s.feather").loc[:"2039-12-31"] + MSL
tf = models.TidalFlat(
    tides=ts,
    land_elev_init=2.6,
    conc_bound=0.3,
    grain_diam=3.0e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate_yr=2e-4,
    comp_rate_yr=4e-3,
    sub_rate_yr=3e-3,
)
tf.run()


Progress:   0%|          | 0/7304 [00:00<?, ?day/s]

----------------------------------------
Starting elevation:       2.600       m
Final elevation:          2.856       m
Elevation change:         25.583     cm
----------------------------------------
Aggradation:              38.934     cm
Degradation:              13.601     cm
----------------------------------------
Runtime:                    05 min 50 s


In [31]:
tf.plot(frac=0.001)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
data.elevation + MSL

datetime
2020-01-01 00:00:00+06:00    0.355046
2020-01-01 00:00:30+06:00    0.362176
2020-01-01 00:01:00+06:00    0.369318
2020-01-01 00:01:30+06:00    0.376474
2020-01-01 00:02:00+06:00    0.383643
                               ...   
2069-12-31 23:57:30+06:00    3.357702
2069-12-31 23:58:00+06:00    3.378441
2069-12-31 23:58:30+06:00    3.399154
2069-12-31 23:59:00+06:00    3.419838
2069-12-31 23:59:30+06:00    3.440493
Name: elevation, Length: 52597440, dtype: float64

In [39]:
ts

datetime
2020-01-01 00:00:00+06:00    0.355044
2020-01-01 00:00:30+06:00    0.362173
2020-01-01 00:01:00+06:00    0.369316
2020-01-01 00:01:30+06:00    0.376472
2020-01-01 00:02:00+06:00    0.383640
                               ...   
2039-12-31 23:57:30+06:00    2.393360
2039-12-31 23:58:00+06:00    2.396914
2039-12-31 23:58:30+06:00    2.400437
2039-12-31 23:59:00+06:00    2.403930
2039-12-31 23:59:30+06:00    2.407393
Freq: 30S, Name: elevation, Length: 21038400, dtype: float64

In [38]:
plt.close("all")
df = tf.results.set_index("datetime")
df["land_elev_delta"] = df.land_elev.diff()
df["land_elev_delta_yearly_avg"] = df.land_elev_delta.rolling(window=pd.offsets.Day(365)).sum()
df["land_elev_delta_yearly_avg"].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime'>

In [56]:
ts = data.elevation.loc[:"2039-12-31"] + MSL
freq = pd.infer_freq(ts.index)
index = pd.DatetimeIndex(df1.index, freq=freq)
ts = pd.Series(data=ts.values, index=index)
tf = models.TidalFlat(
    tides=ts,
    land_elev_init=2.6,
    conc_bound=0.3,
    grain_diam=3.0e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate_yr=2e-4,
    comp_rate_yr=4e-3,
    sub_rate_yr=3e-3,
)
tf.run()


Progress:   0%|          | 0/7304 [00:00<?, ?day/s]

----------------------------------------
Starting elevation:       2.600       m
Final elevation:          3.009       m
Elevation change:         40.919     cm
----------------------------------------
Aggradation:              54.218     cm
Degradation:              13.599     cm
----------------------------------------
Runtime:                    07 min 24 s


In [57]:
plt.close("all")
df = tf.results.set_index("datetime")
df["land_elev_delta"] = df.land_elev.diff()
df["land_elev_delta_yearly_avg"] = df.land_elev_delta.rolling(window=pd.offsets.Day(365)).sum()
df["land_elev_delta_yearly_avg"].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime'>